# Votre base devra contenir ces différentes tables :

Une table appelée population, contenant la population de chaque pays pour 2013. Elle devra contenir 4 colonnes : pays, code_pays, annee, population.

In [1]:
import sqlite3 as sql
import pandas as pd 
import numpy as np
from IPython.display import display

In [2]:
database = "Projet3_Etude_Sante"
cnn=sql.connect(database)

In [3]:
print(cnn)

In [4]:
cursor = cnn.cursor()

In [5]:
cnn

In [6]:

# Crée un  DataFrame
##########################

p2pop_csv = pd.read_csv("Fr_population.csv")

p2pop = pd.DataFrame(p2pop_csv, columns=['Code zone', 'Année', 'Zone', 'Valeur'])

col_names = {
    'Zone': 'Pays',
    'Code zone': 'Code_Pays1',
    'Année': 'year_1',
    'Valeur': 'population'
}

p2pop.rename(columns=col_names, inplace=True)

p2pop['population'] = p2pop['population'] * 1000

p2pop = p2pop[p2pop['Pays'] != 'Chine'].copy()


In [7]:
p2pop

,Code_Pays1,year_1,Pays,population
0,2,2013,Afghanistan,30552000
1,202,2013,Afrique du Sud,52776000
2,3,2013,Albanie,3173000
3,4,2013,Algérie,39208000
4,79,2013,Allemagne,82727000
...,...,...,...,...
170,236,2013,Venezuela (République bolivarienne du),30405000
171,237,2013,Viet Nam,91680000
172,249,2013,Yémen,24407000
173,251,2013,Zambie,14539000


In [8]:
# Cree la table  et importer le dataframe
####################################################

the_table = 'population'

cursor.execute(f"""
            DROP TABLE IF EXISTS {the_table}
            """)

requete = f"""
        CREATE TABLE {the_table} (
        Code_Pays1 SMALLINT UNSIGNED NOT NULL,
        year_1 SMALLINT UNSIGNED NOT NULL,
        Pays VARCHAR(60) NOT NULL,
        population INT UNSIGNED,
        PRIMARY KEY (Code_Pays1, year_1)
        )
        ENGINE=INNODB
    """



In [9]:
#importer le dataframe
p2pop.to_sql(con=cnn,name=the_table,if_exists='append',index=False)

In [10]:
cursor.execute("SELECT*FROM population ORDER BY Code_Pays1 ")

In [11]:
rows=cursor.fetchall()
for row in rows:
    print(row)

(1, 2013, 'Arménie', 2977000)
(2, 2013, 'Afghanistan', 30552000)
(3, 2013, 'Albanie', 3173000)
(4, 2013, 'Algérie', 39208000)
(7, 2013, 'Angola', 21472000)
(8, 2013, 'Antigua-et-Barbuda', 90000)
(9, 2013, 'Argentine', 41446000)
(10, 2013, 'Australie', 23343000)
(11, 2013, 'Autriche', 8495000)
(12, 2013, 'Bahamas', 377000)
(14, 2013, 'Barbade', 285000)
(16, 2013, 'Bangladesh', 156595000)
(17, 2013, 'Bermudes', 65000)
(19, 2013, 'Bolivie (État plurinational de)', 10671000)
(20, 2013, 'Botswana', 2021000)
(21, 2013, 'Brésil', 200362000)
(23, 2013, 'Belize', 332000)
(25, 2013, 'Îles Salomon', 561000)
(26, 2013, 'Brunéi Darussalam', 418000)
(27, 2013, 'Bulgarie', 7223000)
(28, 2013, 'Myanmar', 53259000)
(32, 2013, 'Cameroun', 22254000)
(33, 2013, 'Canada', 35182000)
(35, 2013, 'Cabo Verde', 499000)
(37, 2013, 'République centrafricaine', 4616000)
(38, 2013, 'Sri Lanka', 21273000)
(39, 2013, 'Tchad', 12825000)
(40, 2013, 'Chili', 17620000)
(41, 2013, 'Chine, continentale', 1385567000)
(44, 2

# Création d'une table "dispo_alim" contenant les informations sur la disponibilité alimentaire des différents produits.Question16
Détermination d'une clé primaire pertinente

In [12]:
##########################
# Cree un dataframe
##########################

df_columns = ['Zone', 'Code zone', 'Année', 'Produit', 'Code Produit', 'Élément', 'Valeur']

# Vegetable products
df_vegetable_products = pd.read_csv("fr_vegetaux.csv")[df_columns]
df_vegetable_products['origine'] = 'vegetal'

# Animal products
df_animal_products = pd.read_csv("fr_animaux.csv")[df_columns]
df_animal_products['origine'] = 'animal'

# Combine the two dataframes
df_list = [df_vegetable_products, df_animal_products]
q16_df = pd.concat(df_list)
q16_df.reset_index(drop=True, inplace=True)

In [13]:
q16_df.to_csv("dispo_alim.csv", index = False)


In [14]:
# Pivot
q16_df = pd.pivot_table(
            q16_df,
            index=['Zone', 'Code zone', 'Année', 'Produit', 'Code Produit', 'origine'],
            values='Valeur',
            columns='Élément',
            aggfunc = np.sum
        )
q16_df.reset_index(inplace=True)


In [15]:
q16_df

Élément,Zone,Code zone,Année,Produit,Code Produit,origine,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),...,Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
0,Afghanistan,2,2013,Abats Comestible,2736,animal,NaN,NaN,5.0,1.72,...,0.77,53.0,NaN,NaN,53.0,NaN,53.0,NaN,NaN,NaN
1,Afghanistan,2,2013,"Agrumes, Autres",2614,vegetal,NaN,NaN,1.0,1.29,...,0.02,41.0,2.0,40.0,39.0,2.0,3.0,NaN,NaN,NaN
2,Afghanistan,2,2013,Aliments pour enfants,2680,vegetal,NaN,NaN,1.0,0.06,...,0.03,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2,2013,Ananas,2618,vegetal,NaN,NaN,0.0,0.00,...,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2,2013,Bananes,2615,vegetal,NaN,NaN,4.0,2.70,...,0.05,82.0,NaN,82.0,82.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15697,Îles Salomon,25,2013,Viande de Suides,2733,animal,NaN,NaN,45.0,4.70,...,1.41,3.0,NaN,0.0,3.0,NaN,2.0,NaN,NaN,NaN
15698,Îles Salomon,25,2013,Viande de Volailles,2734,animal,NaN,NaN,11.0,3.34,...,1.14,2.0,NaN,2.0,2.0,NaN,0.0,NaN,NaN,NaN
15699,Îles Salomon,25,2013,"Viande, Autre",2735,animal,NaN,NaN,0.0,0.06,...,0.04,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15700,Îles Salomon,25,2013,Vin,2655,vegetal,NaN,NaN,0.0,0.07,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [16]:
# renommer les colonnes

new_col_names = {
    'Zone': 'Pays',
    'Code zone': 'Code_Pays1',
    'Année': 'year_2',
    'Produit': 'produit',
    'Code Produit': 'Code_produit1',
    'origine': 'origine',
    'Disponibilité alimentaire (Kcal/personne/jour)': 'kcal_supply',
    'Disponibilité de matière grasse en quantité (g/personne/jour)': 'fat_supply',
    'Disponibilité de protéines en quantité (g/personne/jour)': 'prot_supply',
    'Disponibilité alimentaire en quantité (kg/personne/an)': 'quantity_supply'
}

q16_df.rename(columns=new_col_names, inplace=True)

    

In [17]:
# enlever la chine 
q16_df = q16_df[q16_df['Pays'] != 'Chine'].copy()

In [18]:
# Merge the population
temp_df = pd.DataFrame(p2pop, columns=['year_1', 'Code_Pays1', 'population'])
q17_df = q16_df.merge(temp_df, how='left', left_on=['Code_Pays1', 'year_2'], right_on=['Code_Pays1', 'year_1'])
q17_df.drop_duplicates(inplace=True)



In [19]:
q17_df

,Pays,Code_Pays1,year_2,produit,Code_produit1,origine,Aliments pour animaux,Autres utilisations (non alimentaire),kcal_supply,quantity_supply,...,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,year_1,population
0,Afghanistan,2,2013,Abats Comestible,2736,animal,NaN,NaN,5.0,1.72,...,NaN,NaN,53.0,NaN,53.0,NaN,NaN,NaN,2013,30552000
1,Afghanistan,2,2013,"Agrumes, Autres",2614,vegetal,NaN,NaN,1.0,1.29,...,2.0,40.0,39.0,2.0,3.0,NaN,NaN,NaN,2013,30552000
2,Afghanistan,2,2013,Aliments pour enfants,2680,vegetal,NaN,NaN,1.0,0.06,...,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,2013,30552000
3,Afghanistan,2,2013,Ananas,2618,vegetal,NaN,NaN,0.0,0.00,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2013,30552000
4,Afghanistan,2,2013,Bananes,2615,vegetal,NaN,NaN,4.0,2.70,...,NaN,82.0,82.0,NaN,NaN,NaN,NaN,NaN,2013,30552000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15600,Îles Salomon,25,2013,Viande de Suides,2733,animal,NaN,NaN,45.0,4.70,...,NaN,0.0,3.0,NaN,2.0,NaN,NaN,NaN,2013,561000
15601,Îles Salomon,25,2013,Viande de Volailles,2734,animal,NaN,NaN,11.0,3.34,...,NaN,2.0,2.0,NaN,0.0,NaN,NaN,NaN,2013,561000
15602,Îles Salomon,25,2013,"Viande, Autre",2735,animal,NaN,NaN,0.0,0.06,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2013,561000
15603,Îles Salomon,25,2013,Vin,2655,vegetal,NaN,NaN,0.0,0.07,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2013,561000


In [20]:
# Cree une nouvelle colonne
q17_df['tones_supply'] = q17_df['quantity_supply'] * q17_df['population'] / 1000



In [21]:
#virer  quantity supply
q17_df.drop(axis=1, labels=['quantity_supply'], inplace=True)


In [22]:
# nettoyer
q17_df = q17_df[['Code_Pays1', 'Code_produit1', 'year_2', 'Pays', 'population', 'produit', 'origine', 'tones_supply', 'kcal_supply', 'fat_supply', 'prot_supply']]


In [23]:
####################################################
# Create the MySQL Table and import the dataframe
####################################################

the_tables = 'dispo_alim'

cursor.execute(f"""
            DROP TABLE IF EXISTS {the_tables}
            """)

the_request = f"""
        CREATE TABLE {the_table} (
        Code_pays1 SMALLINT UNSIGNED NOT NULL,
        Code_produit1 SMALLINT UNSIGNED NOT NULL,
        year_2 SMALLINT UNSIGNED NOT NULL,
        Pays VARCHAR(60) NOT NULL,
        population INT UNSIGNED,
        produit VARCHAR(60) NOT NULL,
        origine VARCHAR(60),
        tones_supply FLOAT(12,2),
        kcal_supply FLOAT(12,2),
        fat_supply FLOAT(12,2),
        prot_supply FLOAT(12,2),
        PRIMARY KEY (Code_pays1,Code_produit1, year_2)
        )
        ENGINE=INNODB
    """

In [24]:
#importer le dataframe
q17_df.to_sql(con=cnn,name=the_tables,if_exists='append',index=False)

In [25]:
cursor.execute("SELECT*FROM dispo_alim ORDER BY Code_Pays1")

In [26]:
rows=cursor.fetchall()
for row in rows:
    print(row)

(1, 2736, 2013, 'Arménie', 2977000, 'Abats Comestible', 'animal', 13664.43, 14.0, 0.39, 2.25)
(1, 2614, 2013, 'Arménie', 2977000, 'Agrumes, Autres', 'vegetal', 1369.42, 0.0, 0.0, 0.01)
(1, 2659, 2013, 'Arménie', 2977000, 'Alcool, non Comestible', 'vegetal', None, None, None, None)
(1, 2680, 2013, 'Arménie', 2977000, 'Aliments pour enfants', 'vegetal', 476.32, 2.0, 0.01, 0.07)
(1, 2618, 2013, 'Arménie', 2977000, 'Ananas', 'vegetal', 476.32, 0.0, None, 0.0)
(1, 2769, 2013, 'Arménie', 2977000, 'Animaux Aquatiques Autre', 'animal', 0.0, 0.0, 0.0, 0.0)
(1, 2556, 2013, 'Arménie', 2977000, 'Arachides Decortiquees', 'vegetal', 1190.8, 6.0, 0.55, 0.29)
(1, 2516, 2013, 'Arménie', 2977000, 'Avoine', 'vegetal', 1101.49, 2.0, 0.03, 0.09)
(1, 2615, 2013, 'Arménie', 2977000, 'Bananes', 'vegetal', 2292.29, 1.0, 0.01, 0.01)
(1, 2616, 2013, 'Arménie', 2977000, 'Bananes plantains', 'vegetal', 7591.349999999999, 5.0, 0.02, 0.06)
(1, 2740, 2013, 'Arménie', 2977000, 'Beurre, Ghee', 'animal', 5239.52, 35.0, 

(68, 2516, 2013, 'France', 64291000, 'Avoine', 'vegetal', 36645.87, 3.0, 0.06, 0.13)
(68, 2615, 2013, 'France', 64291000, 'Bananes', 'vegetal', 298310.24, 8.0, 0.01, 0.1)
(68, 2616, 2013, 'France', 64291000, 'Bananes plantains', 'vegetal', 38574.6, 1.0, 0.0, 0.01)
(68, 2740, 2013, 'France', 64291000, 'Beurre, Ghee', 'animal', 513042.18, 157.0, 17.72, 0.2)
(68, 2656, 2013, 'France', 64291000, 'Bière', 'vegetal', 1500551.94, 31.0, None, 0.32)
(68, 2511, 2013, 'France', 64291000, 'Blé', 'vegetal', 6971073.13, 811.0, 4.2, 25.99)
(68, 2658, 2013, 'France', 64291000, 'Boissons Alcooliques', 'vegetal', 145297.66, 18.0, None, None)
(68, 2657, 2013, 'France', 64291000, 'Boissons Fermentés', 'vegetal', 500183.98, 10.0, None, 0.02)
(68, 2630, 2013, 'France', 64291000, 'Café', 'vegetal', 358743.78, 7.0, None, 0.8)
(68, 2766, 2013, 'France', 64291000, 'Cephalopodes', 'animal', 30859.68, 1.0, 0.01, 0.18)
(68, 2612, 2013, 'France', 64291000, 'Citrons & Limes', 'vegetal', 136296.92, 1.0, 0.02, 0.02)
(

(131, 2633, 2013, 'Malaisie', 29717000, 'Feve de Cacao', 'vegetal', 9212.27, 3.0, 0.25, 0.05)
(131, 2625, 2013, 'Malaisie', 29717000, 'Fruits, Autres', 'vegetal', 498651.26000000007, 21.0, 0.14, 0.23)
(131, 2642, 2013, 'Malaisie', 29717000, 'Girofles', 'vegetal', 891.51, 0.0, 0.01, 0.0)
(131, 2558, 2013, 'Malaisie', 29717000, 'Graines Colza/Moutarde', 'vegetal', 0.0, 0.0, 0.0, 0.0)
(131, 2559, 2013, 'Malaisie', 29717000, 'Graines de coton', 'vegetal', None, None, None, None)
(131, 2557, 2013, 'Malaisie', 29717000, 'Graines de tournesol', 'vegetal', None, None, None, None)
(131, 2737, 2013, 'Malaisie', 29717000, 'Graisses Animales Crue', 'animal', 20504.73, 14.0, 1.47, 0.08)
(131, 2546, 2013, 'Malaisie', 29717000, 'Haricots', 'vegetal', 39820.78, 12.0, 0.05, 0.79)
(131, 2586, 2013, 'Malaisie', 29717000, 'Huil Plantes Oleif Autr', 'vegetal', 28825.49, 23.0, 2.65, 0.0)
(131, 2575, 2013, 'Malaisie', 29717000, 'Huile Graines de Coton', 'vegetal', 0.0, 0.0, 0.01, None)
(131, 2572, 2013, 'Mal

(188, 2658, 2013, 'Saint-Kitts-et-Nevis', 54000, 'Boissons Alcooliques', 'vegetal', 159.84, 24.0, None, None)
(188, 2657, 2013, 'Saint-Kitts-et-Nevis', 54000, 'Boissons Fermentés', 'vegetal', 66.96, 2.0, None, None)
(188, 2630, 2013, 'Saint-Kitts-et-Nevis', 54000, 'Café', 'vegetal', 31.86, 1.0, None, 0.1)
(188, 2766, 2013, 'Saint-Kitts-et-Nevis', 54000, 'Cephalopodes', 'animal', 1.08, 0.0, 0.0, 0.01)
(188, 2612, 2013, 'Saint-Kitts-et-Nevis', 54000, 'Citrons & Limes', 'vegetal', 0.0, 0.0, None, None)
(188, 2560, 2013, 'Saint-Kitts-et-Nevis', 54000, 'Coco (Incl Coprah)', 'vegetal', 699.84, 65.0, 6.19, 0.61)
(188, 2765, 2013, 'Saint-Kitts-et-Nevis', 54000, 'Crustacés', 'animal', 131.22, 3.0, 0.03, 0.53)
(188, 2743, 2013, 'Saint-Kitts-et-Nevis', 54000, 'Crème', 'animal', 0.0, 0.0, 0.0, 0.0)
(188, 2520, 2013, 'Saint-Kitts-et-Nevis', 54000, 'Céréales, Autres', 'vegetal', 139.86, 21.0, 0.05, 0.56)
(188, 2543, 2013, 'Saint-Kitts-et-Nevis', 54000, 'Edulcorants Autres', 'vegetal', 576.18, 51.0, 

(251, 2735, 2013, 'Zambie', 14539000, 'Viande, Autre', 'animal', 38528.35, 8.0, 0.22, 1.31)
(251, 2655, 2013, 'Zambie', 14539000, 'Vin', 'vegetal', 145.39, 0.0, None, None)
(251, 2645, 2013, 'Zambie', 14539000, 'Épices, Autres', 'vegetal', 2907.8, 2.0, 0.08, 0.06)
(255, 2736, 2013, 'Belgique', 11104000, 'Abats Comestible', 'animal', 36421.12, 10.0, 0.32, 1.66)
(255, 2614, 2013, 'Belgique', 11104000, 'Agrumes, Autres', 'vegetal', 333.12, 0.0, 0.0, 0.0)
(255, 2659, 2013, 'Belgique', 11104000, 'Alcool, non Comestible', 'vegetal', None, None, None, None)
(255, 2680, 2013, 'Belgique', 11104000, 'Aliments pour enfants', 'vegetal', None, None, None, None)
(255, 2618, 2013, 'Belgique', 11104000, 'Ananas', 'vegetal', 29758.72, 4.0, 0.01, 0.02)
(255, 2769, 2013, 'Belgique', 11104000, 'Animaux Aquatiques Autre', 'animal', 666.24, 0.0, 0.0, 0.01)
(255, 2556, 2013, 'Belgique', 11104000, 'Arachides Decortiquees', 'vegetal', 2664.96, 4.0, 0.33, 0.17)
(255, 2516, 2013, 'Belgique', 11104000, 'Avoine', 

# Création d'une table "equilibre_prod" contenant toutes les informations sur la transformation et l'exploitation des différents produits.Détermination d'une clé primaire pertinente.
# Question17

In [27]:
qd18_df= pd.read_csv("dispo_alim.csv")

In [28]:
qd18_df

,Zone,Code zone,Année,Produit,Code Produit,Élément,Valeur,origine
0,Afghanistan,2,2013,Blé,2511,Production,5169.0,vegetal
1,Afghanistan,2,2013,Blé,2511,Importations - Quantité,1173.0,vegetal
2,Afghanistan,2,2013,Blé,2511,Variation de stock,-350.0,vegetal
3,Afghanistan,2,2013,Blé,2511,Disponibilité intérieure,5992.0,vegetal
4,Afghanistan,2,2013,Blé,2511,Semences,322.0,vegetal
...,...,...,...,...,...,...,...,...
142032,Zimbabwe,181,2013,Plantes Aquatiques,2775,Nourriture,0.0,animal
142033,Zimbabwe,181,2013,Plantes Aquatiques,2775,Disponibilité alimentaire en quantité (kg/pers...,0.0,animal
142034,Zimbabwe,181,2013,Plantes Aquatiques,2775,Disponibilité alimentaire (Kcal/personne/jour),0.0,animal
142035,Zimbabwe,181,2013,Plantes Aquatiques,2775,Disponibilité de protéines en quantité (g/pers...,0.0,animal


In [29]:
qd18_df=pd.pivot_table(qd18_df, index=['Code zone', 'Code Produit', 'Année', 'Zone', 'Produit'],
            values='Valeur',
            columns='Élément',
            aggfunc = np.sum,
        )
qd18_df.reset_index(inplace=True)

In [30]:
qd18_df

Élément,Code zone,Code Produit,Année,Zone,Produit,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
0,1,2511,2013,Arménie,Blé,93.0,0.0,1024.0,130.60,3.60,30.52,554.0,1.0,361.0,389.0,32.0,312.0,30.0,10.0,-118.0
1,1,2513,2013,Arménie,Orge,137.0,26.0,0.0,0.00,0.00,0.00,198.0,0.0,9.0,0.0,15.0,189.0,14.0,7.0,0.0
2,1,2514,2013,Arménie,Maïs,96.0,NaN,0.0,0.03,NaN,0.01,102.0,NaN,82.0,0.0,7.0,21.0,0.0,NaN,NaN
3,1,2515,2013,Arménie,Seigle,1.0,NaN,1.0,0.12,0.00,0.02,1.0,NaN,0.0,0.0,0.0,1.0,0.0,NaN,0.0
4,1,2516,2013,Arménie,Avoine,4.0,NaN,2.0,0.37,0.03,0.09,6.0,NaN,1.0,1.0,0.0,5.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15697,351,2781,2013,Chine,Huiles de Poissons,47.0,80.0,0.0,0.00,0.00,0.00,127.0,13.0,91.0,0.0,NaN,50.0,NaN,NaN,0.0
15698,351,2782,2013,Chine,Huiles de Foie de Poisso,0.0,0.0,0.0,0.00,0.00,0.00,0.0,1.0,1.0,0.0,NaN,0.0,NaN,NaN,0.0
15699,351,2805,2013,Chine,Riz (Eq Blanchi),12117.0,2085.0,797.0,77.45,2.68,14.47,135024.0,565.0,2714.0,109725.0,6406.0,136873.0,4679.0,12.0,-3998.0
15700,351,2848,2013,Chine,Lait - Excl Beurre,1632.0,42.0,59.0,33.18,3.25,3.12,50117.0,233.0,9780.0,47009.0,1433.0,40570.0,NaN,0.0,0.0


In [31]:
# Rename, clean and export CSV

the_columns = {
    'Code zone': 'Code_Pays1',
    'Code Produit': 'Code_Produit',
    'Année': 'year_4',
    'Zone': 'Pays',
    'Produit': 'produit',
    'Aliments pour animaux': 'animal_food',
    'Autres utilisations (non alimentaire)': 'other_uses',
    'Disponibilité intérieure': 'domestic_supply_quantity',
    'Nourriture': 'food',
    'Pertes': 'losses',
    'Semences': 'seed',
    'Traitement': 'processed'
}
qd18_df.rename(columns=the_columns, inplace=True)
qd18_df.fillna(0, inplace=True)
qd18_df = qd18_df[qd18_df['Pays'] != 'Chine'].copy()

In [32]:
qd18_df=qd18_df[['Code_Pays1','Code_Produit','year_4','Pays','produit','animal_food','other_uses','domestic_supply_quantity','food','losses','seed','processed']]

In [33]:
the_tables = 'equilibre_prod'

cursor.execute(f"""
            DROP TABLE IF EXISTS {the_tables}
            """)

the_request = f"""
        CREATE TABLE {the_tables} (
        Code_Pays1 SMALLINT UNSIGNED NOT NULL,
        Code_Produit SMALLINT UNSIGNED NOT NULL,
        year_4 SMALLINT UNSIGNED NOT NULL,
        Pays VARCHAR(60) NOT NULL,
        produit VARCHAR(60) NOT NULL,
        animal_food FLOAT(12,2),
        other_uses FLOAT(12,2),
        domestic_supply_quantity FLOAT(12,2),
        food FLOAT(12,2),
        losses FLOAT(12,2),
        seed FLOAT(12,2),
        processed FLOAT(12,2),
        PRIMARY KEY (Code_Pays1, Code_Produit, year_4)
        )
        ENGINE=INNODB
        """

In [34]:
#importer le dataframe
qd18_df.to_sql(con=cnn,name=the_tables,if_exists='append',index=False)

In [35]:
cursor.execute("SELECT*FROM equilibre_prod ORDER BY Code_pays1 ")

In [36]:
rows=cursor.fetchall()
for row in rows:
    print(row)

(1, 2511, 2013, 'Arménie', 'Blé', 93.0, 0.0, 554.0, 389.0, 32.0, 30.0, 10.0)
(1, 2513, 2013, 'Arménie', 'Orge', 137.0, 26.0, 198.0, 0.0, 15.0, 14.0, 7.0)
(1, 2514, 2013, 'Arménie', 'Maïs', 96.0, 0.0, 102.0, 0.0, 7.0, 0.0, 0.0)
(1, 2515, 2013, 'Arménie', 'Seigle', 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)
(1, 2516, 2013, 'Arménie', 'Avoine', 4.0, 0.0, 6.0, 1.0, 0.0, 0.0, 0.0)
(1, 2517, 2013, 'Arménie', 'Millet', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(1, 2518, 2013, 'Arménie', 'Sorgho', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(1, 2520, 2013, 'Arménie', 'Céréales, Autres', 1.0, 12.0, 21.0, 6.0, 1.0, 1.0, 0.0)
(1, 2531, 2013, 'Arménie', 'Pommes de Terre', 225.0, 16.0, 672.0, 134.0, 96.0, 123.0, 78.0)
(1, 2532, 2013, 'Arménie', 'Manioc', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(1, 2534, 2013, 'Arménie', 'Racines nda', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(1, 2537, 2013, 'Arménie', 'Sucre, betterave', 1.0, 0.0, 55.0, 0.0, 0.0, 0.0, 54.0)
(1, 2542, 2013, 'Arménie', 'Sucre Eq Brut', 0.0, 0.0, 93.0, 98.0, 0.0, 0.0, 

(56, 2769, 2013, 'République dominicaine', 'Animaux Aquatiques Autre', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(56, 2775, 2013, 'République dominicaine', 'Plantes Aquatiques', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(56, 2781, 2013, 'République dominicaine', 'Huiles de Poissons', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(56, 2782, 2013, 'République dominicaine', 'Huiles de Foie de Poisso', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(56, 2805, 2013, 'République dominicaine', 'Riz (Eq Blanchi)', 0.0, 0.0, 604.0, 546.0, 16.0, 11.0, 31.0)
(56, 2848, 2013, 'République dominicaine', 'Lait - Excl Beurre', 0.0, 0.0, 801.0, 767.0, 34.0, 0.0, 0.0)
(56, 2899, 2013, 'République dominicaine', 'Miscellanees', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(57, 2511, 2013, 'Bélarus', 'Blé', 1339.0, 25.0, 2324.0, 566.0, 82.0, 133.0, 179.0)
(57, 2513, 2013, 'Bélarus', 'Orge', 1250.0, 13.0, 1699.0, 40.0, 24.0, 220.0, 152.0)
(57, 2514, 2013, 'Bélarus', 'Maïs', 1129.0, 0.0, 1214.0, 5.0, 54.0, 25.0, 0.0)
(57, 2515, 2013, 'Bélarus', 'Seigle',

(110, 2543, 2013, 'Japon', 'Edulcorants Autres', 0.0, 0.0, 1412.0, 1337.0, 0.0, 0.0, 75.0)
(110, 2546, 2013, 'Japon', 'Haricots', 2.0, 0.0, 183.0, 174.0, 4.0, 3.0, 0.0)
(110, 2547, 2013, 'Japon', 'Pois', 0.0, 0.0, 16.0, 15.0, 0.0, 0.0, 0.0)
(110, 2549, 2013, 'Japon', 'Légumineuses Autres', 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0)
(110, 2551, 2013, 'Japon', 'Noix', 0.0, 0.0, 242.0, 239.0, 3.0, 0.0, 0.0)
(110, 2555, 2013, 'Japon', 'Soja', 104.0, 0.0, 3009.0, 933.0, 55.0, 6.0, 1911.0)
(110, 2556, 2013, 'Japon', 'Arachides Decortiquees', 1.0, 0.0, 93.0, 90.0, 0.0, 0.0, 1.0)
(110, 2557, 2013, 'Japon', 'Graines de tournesol', 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0)
(110, 2558, 2013, 'Japon', 'Graines Colza/Moutarde', 0.0, 0.0, 2416.0, 11.0, 10.0, 0.0, 2396.0)
(110, 2559, 2013, 'Japon', 'Graines de coton', 86.0, 0.0, 107.0, 0.0, 0.0, 0.0, 21.0)
(110, 2560, 2013, 'Japon', 'Coco (Incl Coprah)', 0.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0)
(110, 2561, 2013, 'Japon', 'Sésame', 0.0, 0.0, 141.0, 46.0, 1.0, 0.0, 94.0)

(165, 2618, 2013, 'Pakistan', 'Ananas', 0.0, 0.0, 6.0, 6.0, 0.0, 0.0, 0.0)
(165, 2619, 2013, 'Pakistan', 'Dattes', 0.0, 0.0, 434.0, 416.0, 18.0, 0.0, 0.0)
(165, 2620, 2013, 'Pakistan', 'Raisin', 0.0, 0.0, 98.0, 93.0, 5.0, 0.0, 0.0)
(165, 2625, 2013, 'Pakistan', 'Fruits, Autres', 0.0, 0.0, 2632.0, 2567.0, 66.0, 0.0, 0.0)
(165, 2630, 2013, 'Pakistan', 'Café', 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0)
(165, 2633, 2013, 'Pakistan', 'Feve de Cacao', 0.0, 0.0, 9.0, 9.0, 0.0, 0.0, 0.0)
(165, 2635, 2013, 'Pakistan', 'Thé', 0.0, 0.0, 119.0, 116.0, 2.0, 0.0, 0.0)
(165, 2640, 2013, 'Pakistan', 'Poivre', 0.0, 0.0, 6.0, 6.0, 0.0, 0.0, 0.0)
(165, 2641, 2013, 'Pakistan', 'Piments', 0.0, 0.0, 145.0, 142.0, 3.0, 0.0, 0.0)
(165, 2642, 2013, 'Pakistan', 'Girofles', 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0)
(165, 2645, 2013, 'Pakistan', 'Épices, Autres', 0.0, 0.0, 124.0, 124.0, 0.0, 0.0, 0.0)
(165, 2655, 2013, 'Pakistan', 'Vin', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(165, 2656, 2013, 'Pakistan', 'Bière', 0.0, 0.0, 4.0, 4.

(225, 2805, 2013, 'Émirats arabes unis', 'Riz (Eq Blanchi)', 2.0, 150.0, 854.0, 526.0, 175.0, 0.0, 0.0)
(225, 2848, 2013, 'Émirats arabes unis', 'Lait - Excl Beurre', 0.0, 2.0, 1260.0, 1240.0, 18.0, 0.0, 0.0)
(225, 2899, 2013, 'Émirats arabes unis', 'Miscellanees', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(226, 2511, 2013, 'Ouganda', 'Blé', 0.0, 0.0, 396.0, 381.0, 14.0, 1.0, 0.0)
(226, 2513, 2013, 'Ouganda', 'Orge', 10.0, 0.0, 14.0, 1.0, 0.0, 0.0, 7.0)
(226, 2514, 2013, 'Ouganda', 'Maïs', 300.0, 1.0, 2858.0, 1839.0, 299.0, 33.0, 385.0)
(226, 2515, 2013, 'Ouganda', 'Seigle', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(226, 2516, 2013, 'Ouganda', 'Avoine', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(226, 2517, 2013, 'Ouganda', 'Millet', 0.0, 0.0, 228.0, 184.0, 7.0, 4.0, 32.0)
(226, 2518, 2013, 'Ouganda', 'Sorgho', 15.0, 0.0, 250.0, 83.0, 15.0, 11.0, 125.0)
(226, 2520, 2013, 'Ouganda', 'Céréales, Autres', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(226, 2531, 2013, 'Ouganda', 'Pommes de Terre', 0.0, 0.0, 185.0, 148.0,

# Création d'une table "sous_nutrition" contenant les informations sur le nombre de personnes en sous-nutrition par pays et par année.
# Détermination d'une clé primaire pertinente

In [37]:
q19_df = pd.read_csv("fr_sousalimentation.csv")[['Code zone', 'Année', 'Zone', 'Valeur']]


In [38]:
the_columns = {
    'Code zone': 'Code_pays',
    'Zone': 'Pays',
    'Code année': 'Année',
    'Valeur': 'nb_personnes'
    
}

q19_df.rename(columns=the_columns, inplace=True)

In [39]:
q19_df

,Code_pays,Année,Pays,nb_personnes
0,2,2012-2014,Afghanistan,7.9
1,2,2013-2015,Afghanistan,8.8
2,2,2014-2016,Afghanistan,9.6
3,2,2015-2017,Afghanistan,10.2
4,2,2016-2018,Afghanistan,10.6
...,...,...,...,...
1015,181,2012-2014,Zimbabwe,6.6
1016,181,2013-2015,Zimbabwe,7.2
1017,181,2014-2016,Zimbabwe,7.8
1018,181,2015-2017,Zimbabwe,8.2


In [40]:
#Aprés de multiples test à changer le type de Variable en float,j'ai pu voir que le problème
#était dans la ligne qui contenait Barbade
#Nous remplacons la la valeur <0.1 par 0 grace à la fonction.replace


q19_df['nb_personnes']=q19_df['nb_personnes'].replace('<0.1',0)


In [41]:
#J'effectue ensuite un Cast avec la fonction to_numeric permet de convertir mon str en float
q19_df['nb_personnes']=pd.to_numeric(q19_df['nb_personnes'],downcast="float")


In [42]:
#Je multiplie les données de la colonnes valeur pour les mettre en millions
q19_df['nb_personnes']=q19_df['nb_personnes']*1000000


In [43]:
#J'enleve La Chine du Dataframe pour ne pas fausser mes résultats
q19_df =q19_df[q19_df.Pays != 'Chine'].copy()

In [44]:
#Création d'une fonction qui me permettra aprés d'isolé l'année 2013
def format_année(Code_année):
    if Code_année == '2011-2013':
        return 2012

    elif Code_année== '2012-2014':
        return 2013
    
    elif Code_année== '2013-2015':
        return 2014
    
    elif Code_année== '2014-2016':
        return 2015
    
    elif Code_année== '2015-2017':
        return 2016
    
    elif Code_année== '2016-2018':
        return 2017


In [45]:
#Application de la fonction grace à apply
q19_df['Année'] = q19_df['Année'].apply(format_année)

In [46]:
q19_df

,Code_pays,Année,Pays,nb_personnes
0,2,2013,Afghanistan,7.900000e+06
1,2,2014,Afghanistan,8.800000e+06
2,2,2015,Afghanistan,9.600000e+06
3,2,2016,Afghanistan,1.020000e+07
4,2,2017,Afghanistan,1.060000e+07
...,...,...,...,...
1015,181,2013,Zimbabwe,6.600000e+06
1016,181,2014,Zimbabwe,7.200000e+06
1017,181,2015,Zimbabwe,7.800000e+06
1018,181,2016,Zimbabwe,8.200000e+06


In [47]:
the_tables = 'sous_nutrition'

cursor.execute(f"""
            DROP TABLE IF EXISTS {the_tables}
            """)

the_request = f"""
        CREATE TABLE {the_tables} (
        Code_pays SMALLINT UNSIGNED NOT NULL,
        année SMALLINT UNSIGNED NOT NULL,
        Pays VARCHAR(60) NOT NULL,
        nb_personnes INT,
        PRIMARY KEY (Code_pays, Annee)
        )
        ENGINE=INNODB
        """

In [48]:
#importer le dataframe
q19_df.to_sql(con=cnn,name=the_tables,if_exists='append',index=False)

In [49]:
cursor.execute("SELECT*FROM sous_nutrition ORDER BY Code_pays ")

In [50]:
rows=cursor.fetchall()
for row in rows:
    print(row)

(1, 2013, 'Arménie', 100000.00149011612)
(1, 2014, 'Arménie', 100000.00149011612)
(1, 2015, 'Arménie', 100000.00149011612)
(1, 2016, 'Arménie', 100000.00149011612)
(1, 2017, 'Arménie', 100000.00149011612)
(2, 2013, 'Afghanistan', 7900000.095367432)
(2, 2014, 'Afghanistan', 8800000.190734863)
(2, 2015, 'Afghanistan', 9600000.381469727)
(2, 2016, 'Afghanistan', 10199999.809265137)
(2, 2017, 'Afghanistan', 10600000.381469727)
(3, 2013, 'Albanie', 200000.00298023224)
(3, 2014, 'Albanie', 200000.00298023224)
(3, 2015, 'Albanie', 200000.00298023224)
(3, 2016, 'Albanie', 200000.00298023224)
(3, 2017, 'Albanie', 200000.00298023224)
(4, 2013, 'Algérie', 1700000.0476837158)
(4, 2014, 'Algérie', 1600000.023841858)
(4, 2015, 'Algérie', 1600000.023841858)
(4, 2016, 'Algérie', 1600000.023841858)
(4, 2017, 'Algérie', 1600000.023841858)
(5, 2013, 'Samoa américaines', None)
(5, 2014, 'Samoa américaines', None)
(5, 2015, 'Samoa américaines', None)
(5, 2016, 'Samoa américaines', None)
(5, 2017, 'Samoa am

# Question 19 : Écrivez les requêtes SQL permettant de connaître…

Les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, puis en termes de kcal par habitant.

In [51]:
requete = """
                SELECT year_2 as year, Pays, SUM(prot_supply / 1000 * 365) as prot_supply_par_prs
                FROM dispo_alim
                WHERE year='2013'
                GROUP BY year, Pays
                ORDER BY prot_supply_par_prs DESC
                LIMIT 20
                """

In [52]:
cursor.execute(requete)

In [53]:
df_20 = pd.DataFrame(columns=['year', 'Pays', 'prot_supply_par_prs'])

In [54]:
if isinstance(rows, list):

    #Analyser la liste
    rows=cursor.fetchall()
    for row in rows:
        df_20= df_20.append({
            'year': row[0],
            'Pays': row[1],
            'prot_supply_par_prs': row[2]
        },
        ignore_index=True
        )
         # afficher le dataframe
    display(df_20)
    
else:
    print(row)
    

,year,Pays,prot_supply_par_prs
0,2013,Islande,48.56690
1,2013,Chine - RAS de Hong-Kong,47.11055
2,2013,Israël,46.72000
3,2013,Lituanie,45.39140
4,2013,Maldives,44.64680
5,2013,Finlande,42.90940
6,2013,Luxembourg,41.47860
7,2013,Monténégro,40.84350
8,2013,Pays-Bas,40.68290
9,2013,Albanie,40.65005


# Pour l'année 2013, les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de kcal  par habitant.

In [55]:
requetes = """
                SELECT year_2 as year, Pays, SUM(kcal_supply * 365) as kcal_supply_par_prs
                FROM dispo_alim
                WHERE year = 2013
                GROUP BY year, Pays
                ORDER BY kcal_supply_par_prs DESC
                LIMIT 10
                """

In [56]:
cursor.execute(requetes)

In [57]:
df_21 = pd.DataFrame(columns=['year', 'Pays', 'kcal_supply_par_prs'])

In [58]:
if isinstance(rows, list):

    #Analyser la liste
    rows=cursor.fetchall()
    for row in rows:
        df_21= df_21.append({
            'year': row[0],
            'Pays': row[1],
            'kcal_supply_par_prs': row[2]
        },
        ignore_index=True
        )
         # afficher le dataframe
    display(df_21)
    
else:
    print(row)
    

,year,Pays,kcal_supply_par_prs
0,2013,Autriche,1376050.0
1,2013,Belgique,1364005.0
2,2013,Turquie,1353420.0
3,2013,États-Unis d'Amérique,1343930.0
4,2013,Israël,1317650.0
5,2013,Irlande,1314730.0
6,2013,Italie,1305970.0
7,2013,Luxembourg,1292100.0
8,2013,Égypte,1284070.0
9,2013,Allemagne,1278595.0


In [59]:
cnn.commit()

# Identifier les 10 pays ayant le plus faible ratio disponibilité alimentaire / habitant en termes de de protéïnes.


In [60]:
requetes = """
                SELECT year_2 as year, Pays, SUM(prot_supply/1000*365) as prot_supply_par_prs
                FROM dispo_alim
                WHERE year='2013'
                GROUP BY Pays
                ORDER BY prot_supply_par_prs ASC
                LIMIT 10
                """

In [61]:
cursor.execute(requetes)

In [62]:
df_22 = pd.DataFrame(columns=['year', 'Pays', 'prot_supply_par_prs'])

In [63]:

if isinstance(rows, list):
   
    rows=cursor.fetchall()
    for row in rows:
        df_22 = df_22.append({
            'year': row[0],
            'Pays': row[1],
            'prot_supply_par_prs': row[2]
        },
        ignore_index=True
        )
        
   
    display(df_22)


else:
    print(row)

,year,Pays,prot_supply_par_prs
0,2013,Libéria,13.74590
1,2013,Guinée-Bissau,16.07825
2,2013,Mozambique,16.67320
3,2013,République centrafricaine,16.80460
4,2013,Madagascar,17.04185
5,2013,Haïti,17.41050
6,2013,Zimbabwe,17.63680
7,2013,Congo,18.76465
8,2013,Ouganda,19.21360
9,2013,Sao Tomé-et-Principe,19.38150


# La quantité totale (en kg) de produits perdus par pays en 2013.


In [64]:
requete= """
            SELECT year_4 as year ,Pays, SUM(losses*1000000) as losses
            FROM equilibre_prod
            WHERE year='2013'
            GROUP BY year,Pays
            ORDER BY losses DESC
            """

In [65]:
cursor.execute(requete)

In [66]:
df_23 = pd.DataFrame(columns=['year', 'Pays', 'losses'])

In [67]:
if isinstance(rows, list):
   
    rows=cursor.fetchall()
    for row in rows:
        df_23 = df_23.append({
            'year': row[0],
            'Pays': row[1],
            'losses': row[2]
        },
        ignore_index=True
        )
        
   
    display(df_23)


else:
    print(row)

,year,Pays,losses
0,2013,"Chine, continentale",8.957500e+10
1,2013,Brésil,7.591400e+10
2,2013,Inde,5.593000e+10
3,2013,Nigéria,1.985400e+10
4,2013,Indonésie,1.308100e+10
...,...,...,...
169,2013,Bermudes,0.000000e+00
170,2013,Islande,0.000000e+00
171,2013,Kiribati,0.000000e+00
172,2013,Maldives,0.000000e+00


# Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.

In [68]:
my_request = """
                SELECT sn.Année AS year,
                        sn.Pays,
                        sn.nb_personnes,
                        pop.population AS population,
                        (sn.nb_personnes / pop.population) as ratio
                FROM sous_nutrition sn
                
                LEFT JOIN population pop
                    ON sn.Code_pays = pop.Code_Pays1
                        AND sn.Année = pop.year_1
                
                WHERE sn.Année ='2013'
                ORDER BY ratio DESC
                
                LIMIT 10
                """
            

In [69]:
cursor.execute(my_request)

In [70]:
df_24 = pd.DataFrame(columns=['year', 'Pays', 'nb_personnes','population','ratio'])

In [71]:
if isinstance(rows, list):
   
    rows=cursor.fetchall()
    for row in rows:
        df_24 = df_24.append({
            'year': row[0],
            'Pays': row[1],
            'nb_personnes': row[2],
            'population': row[3],
            'ratio': row[4]
        },
        ignore_index=True
        )
        
   
    display(df_24)


else:
    print(row)

,year,Pays,nb_personnes,population,ratio
0,2013,Haïti,5.200000e+06,10317000,0.504022
1,2013,Zambie,7.000000e+06,14539000,0.481464
2,2013,Zimbabwe,6.600000e+06,14150000,0.466431
3,2013,République centrafricaine,2.000000e+06,4616000,0.433276
4,2013,République populaire démocratique de Corée,1.060000e+07,24895000,0.425788
5,2013,Congo,1.800000e+06,4448000,0.404676
6,2013,Tchad,4.900000e+06,12825000,0.382066
7,2013,Angola,8.100000e+06,21472000,0.377235
8,2013,Libéria,1.600000e+06,4294000,0.372613
9,2013,Madagascar,8.200000e+06,22925000,0.357688


# Les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.

In [72]:
my_request = """
                SELECT  produit,
                        SUM(other_uses) as other,
                        SUM(domestic_supply_quantity) as supply,
                        SUM(other_uses) / SUM(domestic_supply_quantity) as ratio
                
                FROM equilibre_prod
                WHERE year_4 = '2013'
                
                GROUP BY produit
                ORDER BY ratio DESC
                LIMIT 10
                        
            """

In [73]:
cursor.execute(my_request)

In [74]:
df_25 = pd.DataFrame(columns=['produit', 'other', 'supply', 'ratio'])

In [75]:
if isinstance(rows, list):
    rows=cursor.fetchall()
    for row in rows:
        df_25 = df_25.append({
            'produit': row[0],
            'other': row[1],
            'supply': row[2],
            'ratio': row[3]
        },
        ignore_index=True
        )
        
    # Print the dataframe
    display(df_25)
    
# The request returns an error message
else:
    print(rows)

,produit,other,supply,ratio
0,"Alcool, non Comestible",21769.0,21741.0,1.001288
1,Huil Plantes Oleif Autr,4733.0,6271.0,0.754744
2,Huile de Palmistes,4625.0,6570.0,0.703957
3,Huile de Palme,35191.0,50409.0,0.698109
4,Girofles,84.0,130.0,0.646154
5,Huile de Colza&Moutarde,12909.0,23416.0,0.551290
6,Graisses Animales Crue,10792.0,22951.0,0.470219
7,Huiles de Poissons,371.0,828.0,0.448068
8,Huile de Soja,17463.0,41961.0,0.416172
9,Plantes Aquatiques,8934.0,23554.0,0.379299


# Question 20 : pour quelques uns des produits identifiés dans cette dernière requête SQL, supposez quelles sont ces "autres utilisations" possibles (recherchez sur internet !)